In [2]:
from cltk.tokenize.word import WordTokenizer
from cltk.stop.latin.stops import STOPS_LIST
from cltk.stem.latin.j_v import JVReplacer
from cltk.stem.lemma import LemmaReplacer
from cltk.tokenize.sentence import TokenizeSentence
from collections import defaultdict

import json
import os
import re
import string


tok = TokenizeSentence('latin').tokenize
stop = STOPS_LIST
wt = WordTokenizer('latin')
j = JVReplacer()
lemmatizer = LemmaReplacer('latin')

In [3]:
class Stupid_Back_Off:
    
    
    def __init__(self,sentence_iterable):
        
        self.bi_gram = defaultdict(lambda: [0,defaultdict(int)])
        self.uni_gram = defaultdict(lambda: [0,defaultdict(int)])
        self.no_gram = [0,defaultdict(int)]
        self.grams = [self.no_gram,self.uni_gram,self.bi_gram]
        
        for sentence in sentence_iterable:
            sent = ['**','**']+sentence+['**STOP**']
            self._update_bigram(sent)
            self._update_unigram(sent)
            self._update_nogram(sent)
            
 
    def _update_nogram(self,sent):
        for i in range(len(sent)):
            self.no_gram[1][(sent[i])] += 1
            self.no_gram[0] += 1

    def _update_unigram(self,sent):
        for i in range(len(sent)-1):
            self.uni_gram[(sent[i])][1][sent[i+1]] += 1
            self.uni_gram[(sent[i])][0] += 1
            
    def _update_bigram(self,sent):
        for i in range(len(sent)-2):
            self.bi_gram[(sent[i],sent[i+1])][1][sent[i+2]] += 1
            self.bi_gram[(sent[i],sent[i+1])][0] += 1
            
            
    def word_proba(self,word,prev=(None),n=2):
        if n == 1 or n == 2:
            return self.grams[n][prev][1][word] / self.grams[n][prev][0]
        else:
            return self.grams[0][1][word] / self.grams[0][0]
        
    def S(self,word,prev_words,i=2):
        """
        Stupid Back-Off model for trigrams.
        P(word | prev_words)
        """
        if i == 0:
            return self.grams[0][1][word] / self.grams[0][0]

        if self.grams[i][prev_words][1][word] > 0:
            return self.grams[i][prev_words][1][word] / self.grams[i][prev_words][0]
        else:
            return .4*self.S(word,prev_words[1:],i-1)

In [4]:
class SentencesIterator(object):
     def __init__(self, dirname):
            self.dirname = dirname
 
     def __iter__(self):
        
        for dname in os.listdir(self.dirname):
            dname = self.dirname + dname
            for fname in os.listdir(dname):
                fname = dname + '/' + fname
                sentences = json.loads(open(fname).read())
                for sentence in sentences:
                    yield sentence

sentences = SentencesIterator('/home/nbangs/Notebooks/machine_translation/latin_corpus_tokenized/')

In [5]:
sb = Stupid_Back_Off(sentences)

In [7]:
sb.S('si',('**','**'))

0.02065723159248861

In [8]:
import dill as pkl

In [10]:
with open('../pkls/latin_lang_model.pk', 'wb') as fout:
    pkl.dump(sb,fout)

In [ ]:
mod = pkl.dumps(sb)

In [11]:
print('done')

done


In [ ]:
os.chdir('../..')

In [ ]:
os.listdir()